In [2]:
%%capture
!pip install diffusers
!pip install transformers scipy ftfy
!pip install "ipywidgets>=7,<8"
!pip install accelerate

In [48]:
import huggingface_hub, diffusers, torch, os, google, itertools, warnings

warnings.filterwarnings('ignore')
google.colab.drive.mount('/content/drive')
WORKING_DIR = "drive/MyDrive/__GGabry/StableDiffusion-utils"

try: os.chdir(WORKING_DIR)
except: pass

assert torch.cuda.get_device_name() == "Tesla T4"
from utils import  StableDiffusionHelper # type: ignore

from env.tokens import HUGGING_FACE_TOKEN
huggingface_hub.hf_api.HfFolder.save_token(HUGGING_FACE_TOKEN)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%capture
model_hub = "stabilityai/stable-diffusion-2-1"

lmsd_scheduler = diffusers.LMSDiscreteScheduler(
  beta_start=0.00085,
    beta_end=0.012,
    beta_schedule="scaled_linear"
)

PIPE = diffusers.StableDiffusionPipeline.from_pretrained(
    pretrained_model_name_or_path = model_hub,
    scheduler = lmsd_scheduler,
    torch_dtype=torch.float16,
    revision="fp16",
    use_auth_token=True,
    safety_checker=None
  ).to("cuda")

STH = StableDiffusionHelper(PIPE)

In [ ]:
prompts = list(itertools.chain.from_iterable([
  ["a bee, Cartoon, Artstation, stylish, digital art"] *3,
  ["a magic bee, anime, artstation, digital art"] * 3
]))


N_STEPS = 100
GUIDANCE_SCALES = {3, 7, 12, 30}

STH = StableDiffusionHelper(PIPE)
imgs = STH.generate_images(prompts, GUIDANCE_SCALES, N_STEPS, w=512, h=512)

In [ ]:
STH.display_images_grid(imgs)
